# Export YOLO-NAS ONNX model and metadata for inference

Follow and execute the notebook instructions below to fetch and export yolo_nas models, change variables where mentioned to export a different model.

> __NOTE__: GPU session is not required to export the model.

## 1. Fetch and install super-gradients

In [ ]:
!git clone https://github.com/Deci-AI/super-gradients.git

We switch to PR #2061 to get the fixes for model URLs.

> __IMPORTANT__: After this installation step the session will ask for restart. Click on restart and **follow from step 2**. No need to run any installation steps below.

In [ ]:
%cd /content/super-gradients/
!git fetch origin pull/2061/head:url_fixes
!git switch url_fixes
!pip3 install -r requirements.txt && python3 -m pip install -e .

## 2. Import model and fetch model weights

In [ ]:
from super_gradients.training import models

Change **model_type** to the required model type *(yolo_nas_s, yolo_nas_m, yolo_nas_l)*

In [ ]:
model_type = "yolo_nas_s"

model = models.get(model_type, pretrained_weights="coco")

## 3. Export model to ONNX

Here the ONNX opset version can be set, currently set to version 11. This is to match the opset used by ONNX runtime compiled with OpenCV 4.6.0.
> __NOTE__: This is not the same as ONNX runtime version

After the step completes the model will be available in the current directory as `yolo_nas_<version>.onnx`.

In [ ]:
model = models.get(model_type, pretrained_weights="coco")
model.eval()
model.prep_model_for_conversion(input_size=(1, 3, 640, 640))
# models.convert_to_onnx(model=model, prep_model_for_conversion_kwargs={"input_size":(1, 3, 640, 640)}, out_path="yolo_nas_s.onnx",
#                        torch_onnx_export_kwargs={"opset_version":14})
model.export(f"{model_type}.onnx", postprocessing=None, preprocessing=None,
             onnx_export_kwargs={"opset_version":11})

## 4. Export metadata

The pre and post processing steps are serialised here into json, this will be used in the inference engine to create the pre and post processing pipelines.

The metadata will be available in current directory as `yolo_nas_<version>-metadata.json`.

In [ ]:
import super_gradients.training.processing as processing

def get_preprocessing_steps(preprocessing):
    if isinstance(preprocessing, processing.StandardizeImage):
        return {"StandardizeImage": {"max_value": preprocessing.max_value}}
    elif isinstance(preprocessing, processing.DetectionRescale):
        return {
            "DetectionRescale": {
                "output_shape": preprocessing.output_shape
            }
        }
    elif isinstance(preprocessing, processing.DetectionLongestMaxSizeRescale):
        return {
            "DetectionLongestMaxSizeRescale": {
              "output_shape": preprocessing.output_shape
            }
        }
    elif isinstance(preprocessing, processing.DetectionBottomRightPadding):
        return {
            "DetectionBottomRightPadding": {
                "pad_value": preprocessing.pad_value,
                "output_shape": preprocessing.output_shape
            }
        }
    elif isinstance(preprocessing, processing.DetectionCenterPadding):
        return {
            "DetectionCenterPadding": {
                "pad_value": preprocessing.pad_value,
                "output_shape": preprocessing.output_shape
            }
        }
    elif isinstance(preprocessing, processing.NormalizeImage):
        return {
            "NormalizeImage": {"mean": preprocessing.mean.tolist(), "std": preprocessing.std.tolist()}
        }
    elif isinstance(preprocessing, processing.ImagePermute):
        return {
            "ImagePermute": {
                "order": preprocessing.permutation
            }
        }
    elif isinstance(preprocessing, processing.ReverseImageChannels):
        return {
            "ReverseImageChannels": True
        }
    else:
        raise NotImplemented("Model have processing steps that haven't been implemented")

def get_postprocessing_steps(postprocessing):
  if isinstance(postprocessing, processing.DetectionLongestMaxSizeRescale):
    return {
        "DetectionLongestMaxSizeRescale": {
            "output_shape": postprocessing.output_shape
        }
    }

# serialise preprocessing
preprocessing_steps = [
    get_preprocessing_steps(st) for st in model._image_processor.processings
]

import numpy as np
dummy = np.random.randint(0, 255, (1000, 800, 3), dtype=np.uint8)

input_shape = np.expand_dims(model._image_processor.preprocess_image(dummy)[0], 0).shape

postprocessing_steps = {
    "NMS": {
        "iou": model._default_nms_iou,
        "conf": model._default_nms_conf
    }
}

# get coco labels
labels = model.get_class_names()

# create metadata object
metadata = {
        "type": model_type,
        "input_shape": input_shape,
        "post_processing": postprocessing_steps,
        "pre_processing": preprocessing_steps,
        "labels": labels,
    }

# export metadata
import json

filename = f"{model_type}-metadata.json"
with open(filename, "w") as f:
  f.write(json.dumps(metadata))

metadata

# 5. Download model and metadata
- `/content/yolo_nas_<version>.onnx`
- `/content/yolo_nas_<version>-metadata.json`

Click the folder icon on the left > right click on the file > click download.